In [1]:
from model import Posicao, Extrato
from views import FundoInvestimento, Acao, FundoImobiliario
#from controllers import MainController
import numpy as np
import pandas as pd 
from datetime import datetime

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
posicao_model = Posicao() 
posicao_model.load_data()
extrato = Extrato(2010, 2020)
fundo_investimento = FundoInvestimento(
    posicao=posicao_model.fis, 
    extrato=extrato.extrato_fis
)
acoes = Acao(posicao=posicao_model.acoes, extrato=extrato.df)
fiis = FundoImobiliario(posicao=posicao_model.fiis, extrato=extrato.df)

/Users/gabriellopes/Documents/Pessoal/dev/finance-app/app/classes/model.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.aportes_fi_hist['Nome'] = self.aportes_fi_hist['Descricao'].apply(self.__map_fi)
/Users/gabriellopes/Documents/Pessoal/dev/finance-app/app/classes/model.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ir_fi_hist['Nome'] = self.ir_fi_hist['Descricao'].apply(self.__map_fi)


**Mapeado todos os FIs**

Verifica se existe FI no extrato não mapeado

In [3]:
#fundo_investimento.extrato[fundo_investimento.extrato['Nome'].str.contains('unknown')].head()
extrato.aportes_fi_hist[extrato.aportes_fi_hist['Nome'] == 'unknown']

,Liq,Mov,Descricao,Valor,Saldo,ano,mes,Nome


## Analise FIs

In [4]:
resumo_ = fundo_investimento.resumo

resumo_.head()

,Nome,Total Bruto,mes,ano,data_posicao,index,aporte,retirada,rendimento_resgatado,rendimento,periodo_cont
0,AZ Quest Multi FIC FIM,0.00,1,2010,2010-01-31,0.00,0.00,0.00,0.00,0.00,0.00
1,AZ Quest Multi FIC FIM,0.00,2,2010,2010-02-28,1.00,0.00,0.00,0.00,0.00,0.00
2,AZ Quest Multi FIC FIM,0.00,3,2010,2010-03-31,2.00,0.00,0.00,0.00,0.00,0.00
3,AZ Quest Multi FIC FIM,0.00,4,2010,2010-04-30,3.00,0.00,0.00,0.00,0.00,0.00
4,AZ Quest Multi FIC FIM,0.00,5,2010,2010-05-31,4.00,0.00,0.00,0.00,0.00,0.00


In [5]:
# um fundo especifico

#display(resumo_[resumo_['Nome'] == 'Azul Quantitativo FIM'])

In [11]:
tmp = resumo_.groupby(['Nome'])\
        .agg(total_aporte=('aporte', 'sum'), 
             total_rendimento=('rendimento', 'sum'), 
             total_retirada=('retirada', 'sum')).reset_index()

tmp['%'] = round((tmp['total_rendimento'] / tmp['total_aporte']) *100, 2)

tmp.sort_values('%', ascending=False)

,Nome,total_aporte,total_rendimento,total_retirada,%
11,XP Long Short FIC FIM,"15,000.00","4,822.42",0.00,32.15
0,AZ Quest Multi FIC FIM,"15,000.00","3,591.84","18,591.84",23.95
6,Mauá Macro FIC FIM,"23,500.00","5,382.28",0.00,22.90
7,Mirae Asset Multimercado,"16,000.00","3,122.71","19,122.71",19.52
3,Bahia AM Maraú FIC de FIM,"73,000.00","12,897.97",0.00,17.67
1,Azul Quantitativo FIM,"15,200.00","2,624.47","17,824.47",17.27
9,Polo Norte I FIC FIM,"30,000.00","3,556.88",0.00,11.86
8,NP Hedge FIC FIM,"5,000.00",144.33,"5,144.33",2.89
2,BNP Paribas Inflação FI RF,"10,300.00",-131.86,"10,168.14",-1.28
5,Icatu Vanguarda Pré-Fixado FIRF,"31,700.00",-934.23,"30,765.77",-2.95


In [7]:
display(resumo_[resumo_['Nome'] == 'XP Absoluto Consumo FIA'])

,Nome,Total Bruto,mes,ano,data_posicao,index,aporte,retirada,rendimento_resgatado,rendimento,periodo_cont
0,XP Absoluto Consumo FIA,0.00,1,2010,2010-01-31,"1,320.00",0.00,0.00,0.00,0.00,0.00
1,XP Absoluto Consumo FIA,0.00,2,2010,2010-02-28,"1,321.00",0.00,0.00,0.00,0.00,0.00
2,XP Absoluto Consumo FIA,0.00,3,2010,2010-03-31,"1,322.00",0.00,0.00,0.00,0.00,0.00
3,XP Absoluto Consumo FIA,0.00,4,2010,2010-04-30,"1,323.00",0.00,0.00,0.00,0.00,0.00
4,XP Absoluto Consumo FIA,0.00,5,2010,2010-05-31,"1,324.00",0.00,0.00,0.00,0.00,0.00
5,XP Absoluto Consumo FIA,0.00,6,2010,2010-06-30,"1,325.00",0.00,0.00,0.00,0.00,0.00
6,XP Absoluto Consumo FIA,0.00,7,2010,2010-07-31,"1,326.00",0.00,0.00,0.00,0.00,0.00
7,XP Absoluto Consumo FIA,0.00,8,2010,2010-08-31,"1,327.00",0.00,0.00,0.00,0.00,0.00
8,XP Absoluto Consumo FIA,0.00,9,2010,2010-09-30,"1,328.00",0.00,0.00,0.00,0.00,0.00
9,XP Absoluto Consumo FIA,0.00,10,2010,2010-10-31,"1,329.00",0.00,0.00,0.00,0.00,0.00


In [8]:
resumo_fi = fundo_investimento.resumo(2010, 2020)

TypeError: 'DataFrame' object is not callable

In [ ]:
#display(resumo_fi[resumo_fi['Nome'] == 'Icatu Vanguarda Pré-Fixado FIRF '].sum())

display(resumo_fi[resumo_fi['Nome'] == 'Icatu Vanguarda Pré-Fixado FIRF '])

In [ ]:
resumo_fi.head(1)

In [ ]:
start = datetime(resumo_fi['ano'].min(), 1, 1)
end = datetime(resumo_fi['ano'].max(), 12, 1)

print('De: {} até {}'.format(start, end))

In [ ]:
resumo_fi['Nome'].unique()

In [ ]:
fundo_investimento.extrato.head()

In [ ]:
display(resumo_fi.sort_values(['ano', 'mes'], ascending=False).head(2))
resumo_fi[resumo_fi['Nome'] == 'Azul Quantitativo FIM']

In [ ]:
df_pos[df_pos['Nome'] == 'Bahia AM Maraú FIC de FIM']

In [ ]:
extrato.aportes_xp